# Ejercicio 1

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import glob
import os

plt.rcParams['figure.figsize'] = (12, 6)
plt.style.use('seaborn')

In [6]:
dir_actual = os.getcwd()
archivos_dump = list(map(lambda x: f'{dir_actual}\{x}', glob.glob('input/dump/*.csv')))

dfs = (pd.read_csv(f) for f in archivos_dump)
df = pd.concat(dfs).drop(columns='Unnamed: 0')
df.columns = ['artist', 'genre', 'song_name', 'lyrics']

# Ejercicio 2

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english', max_features=5000)
vectorizer_fit = vectorizer.fit_transform(df['lyrics'])

words = vectorizer.get_feature_names_out()
freqs = vectorizer_fit.toarray().sum(axis=0)

print(f'Las 5000 palabras mas frecuentes en la base de datos son: {words}')

Las 5000 palabras mas frecuentes en la base de datos son: ['000' '10' '100' ... 'zoo' 'zoom' 'zulu']
